## Setup

### Imports

In [ ]:
import torch
from datetime import datetime
from itertools import count

import stockcore.data as scdata
import stockcore.models as scmodels
import stockcore.environment as scenv
import stockcore.utils as scutils

In [ ]:
is_ipython = scutils.setup_mpl()
device = scutils.get_device()

if is_ipython:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

### Data

In [ ]:
from stocksense.api.data import get_kline_df

endpoint = 'binance'
symbol = 'BTC/USDT'

# df is a DataFrame with columns : "open", "high", "low", "close", "volume"
df = await get_kline_df(endpoint, symbol, since=datetime(2018, 5, 15), until=datetime(2024, 11, 20), timeframe='1h')
df = scdata.data_preprocess(df)

In [ ]:
# Create a TradingEnv environment
env = scenv.make_trading_env(
    name=symbol,
    df=df,  # Your dataset with your custom features
    positions=[-1, 0, 1],  # -1 (=SHORT), 0(=OUT), +1 (=LONG)
    trading_fees=0.01/100,  # 0.01% per stock buy / sell (Binance fees)
    borrow_interest_rate=0.0003/100,  # 0.0003% per timestep (one timestep = 1h here)
)

In [ ]:

device = scutils.get_device()

# Load the model
policy_net = scmodels.NaiveDQN(env).to(device)
policy_net.load_state_dict(torch.load(
    "model.pth", weights_only=True, map_location=device))
policy_net.eval()

In [ ]:
# Test the model
state, info = env.reset()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
for t in tqdm(count(), total=len(df)):
    action = policy_net.act(state)
    observation, reward, terminated, truncated, _ = env.step(action.item())
    state = torch.tensor(
        observation,
        dtype=torch.float32, device=device).unsqueeze(0)
    if terminated or truncated:
        break